In [6]:
import pandas as pd
from sklearn.datasets import make_classification
import seaborn as sns
import matplotlib.pyplot as plt

In [116]:
# Ścieżka do pliku CSV
file_path = "./mushroom-classification/mushrooms.csv"

# Wczytaj dane
data = pd.read_csv(file_path)
# Wyświetl pierwsze kilka wierszy
X = data.drop("class", axis=1)  # 'class' to kolumna docelowa (etykiety)
y = data["class"]
#print(y.unique())
#for feature in X.columns:
#print(y)
#print(X['bruises'].unique())
X_encoded = X.apply(lambda col: col.astype('category').cat.codes)
y_encoded = y.astype('category').cat.codes
for feature in y.unique():
    print(feature)
    print(len(X[(y==feature) & (X["cap-shape"] == "x")]))

print(y)

for column, row in X.iterrows():
    print(row['stalk-color-above-ring'])


p
1708
e
1948
0       p
1       e
2       e
3       p
4       e
       ..
8119    e
8120    e
8121    e
8122    p
8123    e
Name: class, Length: 8124, dtype: object
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
g
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
p
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
g
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w

In [115]:
class MulitnomialNaiveBayesClassifie:

    def __init__(self):
        self.feature_probability={}
        self.conditional_probability={} #domyslnie kazde jest warunkowe wzgldem wartosci z feature_probability.

    def fit(self, X_train, y):
        """
        X_train jest tablica wartosci w naszym zbiorze, zapisany z uzyciem DataFrame'u, bibliotek pandas.
        y jest kolumna cech dla ktorej bedziemy wykonywac predykcje, zapisana przy uzyciu Seris -> kolumna wzieta z DataFrame'a.
        Nie zwraca nic.
        """

        for feature in y.unique():
            self.feature_probability[feature]=len(y[y==feature])/len(y) #mozna poprawc, by przechowywac liczby calkowito liczbowe

        for column in X.columns:
            self.conditional_probability[column]={}
            for conditional_feature in X[column].unique():
                self.conditional_probability[column][conditional_feature]={}
                for feature in y.unique():
                    feature_count=len(y[y==feature])
                    conditional_feature_count=len(X_train[(y==feature) & (X_train[column]==conditional_feature)])
                    self.conditional_probability[column][conditional_feature][feature]=conditional_feature_count/feature_count
                    
    def predict(self, row_to_predict, expected_value):
        """
        row_to_predict: struktura Seris zawierający dane, dla których chcemy wykonać predykcję
        """
        predictions = []

        # Obliczanie prawdopodobieństwa dla każdej klasy
        for feature in self.feature_probability:
            probability = self.feature_probability[feature]

            for column in X_to_predict.columns:
                value = row[column]
                if value in self.conditional_probability[column]:
                    probability *= self.conditional_probability[column][value].get(feature, 0)
                else:
                    probability *= 1e-16  # Brak warunku oznacza male prawdopodobienstwo.  
            probability_per_feature[feature] = probability

            # Wybieranie klasy z najwyższym prawdopodobieństwem
        predicted_feature = max(probability_per_feature, key=probability_per_feature.get)
        predictions.append(predicted_feature)
        print(predicted_feature)
        return predictions
        
    def predict_proba():
        

In [100]:
NBC=MulitnomialNaiveBayesClassifie()
NBC.fit(X, y)
NBC.predict(X)
NBC.feature_probability


p
e
e
p
e
e
e
e
p
e
e
e
e
p
e
e
e
p
p
p
e
p
e
e
e
p
e
e
e
e
e
p
e
e
e
e
e
p
e
e
e
e
e
p
e
e
e
e
e
e
e
e
e
p
p
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
p
e
e
p
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
p
e
e
e
e
e
p
e
p
e
e
e
e
e
e
e
e
e
e
e
e
p
e
e
p
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
p
e
e
e
e
p
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
p
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
p
e
e
e
e
e
e
p
e
e
p
e
e
e
e
e
e
e
e
e
e
e
p
e
e
e
e
e
e
e
p
e
e
e
e
e
e
e
e
e
p
e
e
e
e
e
e
e
p
e
p
e
e
e
e
e
e
e
e
p
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
p
e
e
e
e
e
e
e
e
e
e
e
p
e
e
e
p
e
e
e
e
e
e
e
e
e
e
e
p
e
e
p
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
p
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
p
e
e
e
e
p
e
e
e
e
e
e
e
e
e
e
e
e
e
p
e
e
p
e
e
e
e
e
e
e
e
e
e
e
p
e
e
p
e
e
e
e
p
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
e
p
p
e
e
e
e
e
e
e


{'p': 0.48202855736090594, 'e': 0.517971442639094}

In [38]:
for column in data.columns:
    if column != "class": # Ominięcie kolumny z klasą
        print(f"Cecha: {column}")
        print(data.groupby("class")[column].value_counts(), "\n")

Cecha: cap-shape
class  cap-shape
e      x            1948
       f            1596
       b             404
       k             228
       s              32
p      x            1708
       f            1556
       k             600
       b              48
       c               4
Name: count, dtype: int64 

Cecha: cap-surface
class  cap-surface
e      f              1560
       y              1504
       s              1144
p      y              1740
       s              1412
       f               760
       g                 4
Name: count, dtype: int64 

Cecha: cap-color
class  cap-color
e      n            1264
       g            1032
       w             720
       e             624
       y             400
       p              56
       b              48
       c              32
       r              16
       u              16
p      n            1020
       e             876
       g             808
       y             672
       w             320
       b             120

In [4]:

# Wykres punktowy zapachu (odor) i rozmiaru blaszek (gill-size)
sns.scatterplot(data=data_encoded, x='odor', y='gill-size', hue=data['class'], palette={'e': 'green', 'p': 'red'})
plt.title("Zależność między 'odor' a 'gill-size'")
plt.show()

NameError: name 'sns' is not defined